# SVM 5-fold cross-validation of Wang et al. dataset



In [6]:
import os 
import numpy as np
import pandas as pd
import scanpy as sc
import time as tm
import seaborn as sns
from sklearn.svm import LinearSVC
import rpy2.robjects as robjects
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
import anndata as ad

%matplotlib inline 
sc.set_figure_params(dpi=800)

os.chdir("/path/to/directory/") #adjust path

## Defining the SVM function:
Note: function adapted from Abdelaal et al. (2019)

    

In [7]:
def run_SVM(Data, LabelsPath, CV_RDataPath, OutputDir, GeneOrderPath = "", NumGenes = 0, skipNorm = False):
    '''
    run baseline classifier: SVM
    Wrapper script to run an SVM classifier with a linear kernel on a benchmark dataset with 5-fold cross validation,
    outputs lists of true and predicted cell labels as csv files, as well as computation time.

    Parameters:
    Data : Data matrix, cells-genes matrix with cell unique barcodes as row names and gene names as column names.
    LabelsPath : Cell population annotations file path (.csv).
    CV_RDataPath : Cross validation RData file path (.RData), obtained from Cross_Validation.R function.
    OutputDir : Output directory defining the path of the exported file.
    GeneOrderPath : Gene order file path (.csv) obtained from feature selection, 
        defining the genes order for each cross validation fold, default is NULL.
    NumGenes : Number of genes used in case of feature selection (integer), default is 0.
    skipNorm: skip normalisation step? Default is False, only skip if using e.g. integrated pre-normalised data.
    '''
        
    # read the Rdata file
    robjects.r['load'](CV_RDataPath)

    nfolds = np.array(robjects.r['n_folds'], dtype = 'int')
    tokeep = np.array(robjects.r['Cells_to_Keep'], dtype = 'bool')
    col = np.array(robjects.r['col_Index'], dtype = 'int')
    col = col - 1 
    test_ind = np.array(robjects.r['Test_Idx'], dtype = 'object')
    train_ind = np.array(robjects.r['Train_Idx'], dtype = 'object')

    # read the data
    data = Data
    labels = pd.read_csv(LabelsPath, header=0,index_col=None, sep=',', usecols = col)
    
    labels = labels.iloc[tokeep]
    data = data.iloc[tokeep]
    
    # read the feature file
    if (NumGenes > 0):
        features = pd.read_csv(GeneOrderPath,header=0,index_col=None, sep=',')
    
    # normalise data
    if skipNorm == False:
        data = np.log1p(data)
        
    Classifier = LinearSVC()
            
    tr_time=[]
    ts_time=[]
    truelab = []
    pred = []
        
    for i in range(np.squeeze(nfolds)):
        test_ind_i = np.array(test_ind[i], dtype = 'int') - 1
        train_ind_i = np.array(train_ind[i], dtype = 'int') - 1
    
        train=data.iloc[train_ind_i]
        test=data.iloc[test_ind_i]
        y_train=labels.iloc[train_ind_i]
        y_test=labels.iloc[test_ind_i]
            
        if (NumGenes > 0):
            feat_to_use = features.iloc[0:NumGenes,i]
            train = train.iloc[:,feat_to_use]
            test = test.iloc[:,feat_to_use]
            
        start=tm.time()
        Classifier.fit(train, y_train.values.ravel())
        tr_time.append(tm.time()-start)
                    
        start=tm.time()
        predicted = Classifier.predict(test)
        ts_time.append(tm.time()-start)
            
        truelab.extend(y_test.values)
        pred.extend(predicted)
        
                
    truelab = pd.DataFrame(truelab)
    pred = pd.DataFrame(pred)
        
    tr_time = pd.DataFrame(tr_time)
    ts_time = pd.DataFrame(ts_time)
        
    if (NumGenes == 0):  
        truelab.to_csv(str(OutputDir) + "SVM_True_Labels.csv", index = False)
        pred.to_csv(str(OutputDir) + "SVM_Pred_Labels.csv", index =False)
        tr_time.to_csv(str(OutputDir) + "SVM_Training_Time.csv", index = False)
        ts_time.to_csv(str(OutputDir) + "SVM_Testing_Time.csv", index = False)
    else:
        truelab.to_csv(str(OutputDir) +"SVM_" + str(NumGenes) + "_True_Labels.csv", index = False)
        pred.to_csv(str(OutputDir) +"SVM_" + str(NumGenes) + "_Pred_Labels.csv", index = False)
        tr_time.to_csv(str(OutputDir) +"SVM_" + str(NumGenes) + "_Training_Time.csv", index = False)
        ts_time.to_csv(str(OutputDir) +"SVM_" + str(NumGenes) + "_Testing_Time.csv", index = False)

In [8]:
# Load data:
mtx = sc.read_mtx("./Data/Wang_kc2_count.mtx") #adjust path if needed
cells = pd.read_table("./Data/Wang_kc2_metadata.tsv") #adjust path if needed
genes = pd.read_table("./Data/Wang_kc2_features.tsv", header=None, index_col=0) #adjust path if needed
genes.index.name = None
Wang_dat = ad.AnnData(
    X=mtx.X.T,
    obs=cells,
    var=genes,
)

matrix = pd.DataFrame.sparse.from_spmatrix(Wang_dat.X, index=list(Wang_dat.obs.index.values), columns=list(Wang_dat.var.index.values))

In [9]:
# run functions:
run_SVM(Data = matrix,
        LabelsPath = "./Data/Wang_kc2_labels.csv", #adjust path if needed
        CV_RDataPath = "./Data/CV_folds.RData", #adjust path if needed
        OutputDir = "./Results/CrossVal/", GeneOrderPath = "", NumGenes = 0) #adjust path if needed

In [10]:
# plot SVM

true = pd.read_csv("./Results/CrossVal/SVM_True_Labels.csv") #adjust path if needed
pred = pd.read_csv("./Results/CrossVal/SVM_Pred_Labels.csv") #adjust path if needed

true.columns = ["True"]
pred.columns = ["Predicted"]

# set labels
labels = list(set(np.hstack((true["True"].astype(str).unique(), pred["Predicted"].astype(str).unique()))))

# create confusion matrix
cnf_matrix = pd.DataFrame(confusion_matrix(true["True"], pred["Predicted"], labels=labels), index=labels, columns=labels)
cnf_matrix = cnf_matrix.loc[true["True"].astype(str).unique(), pred["Predicted"].astype(str).unique()]
cnf_matrix = cnf_matrix.div(cnf_matrix.sum(1), axis=0)

cnf_matrix = cnf_matrix.sort_index(axis=1)
cnf_matrix = cnf_matrix.sort_index()

cnf_matrix = cnf_matrix[['basal I', 'basal II', 'basal III', 'basal IV', 'stratum spinosum', 'stratum granulosum']]
cnf_matrix = cnf_matrix.T[['basal I', 'basal II', 'basal III', 'basal IV', 'stratum spinosum', 'stratum granulosum']]
cnf_matrix = cnf_matrix.T

#plot
sns.set(font_scale=1.2)
cm = sns.clustermap(cnf_matrix.T, cmap="Blues", annot=True,fmt='.2%', row_cluster=False,col_cluster=False)
cm.savefig("./Results/CrossVal/Confusion_matrix_SVM.png")